In [167]:
# Função para obter dados semanais do youtube charts

import requests
import json
import pandas as pd
from datetime import datetime, timedelta
from pandas.io.json import json_normalize

# Formatação de data e inicialização dos arquivos de saída.
# A função faz um loop de cada semana, pega as 100 tracks mais tocadas e concatena no arquivo de saída.
# Ao final do loop o arquivo de saída é salvo em um .csv para ser consumido no Tableau.

def get_youtube_charts_data(start_date='20190104', end_date='20230602'):
    date_format = '%Y%m%d'
    current_date = datetime.strptime(start_date, date_format)
    end_date = datetime.strptime(end_date, date_format)
    file_artistas = []
    file_tracks = []

    while current_date <= end_date:
        # Formatação de datas de início e fim da semana, a serem inseridas na URL
        formatted_date = current_date.strftime(date_format)
        week_date = (current_date + timedelta(days=6)).strftime(date_format)
        
        #Parametros da requisição. Os valores de data em 'query' variam conforme a semana selecionada.
        params = {
            "context": {
                "client": {
                    "clientName": "WEB_MUSIC_ANALYTICS",
                    "clientVersion": "0.2",
                    "hl": "pt",
                    "gl": "BR",
                    "experimentIds": [],
                    "experimentsToken": "",
                    "theme": "MUSIC"
                },
                "capabilities": {},
                "request": {
                    "internalExperimentFlags": []
                }
            },
            "browseId": "FEmusic_analytics_charts_home",
            "query": f"chart_params_type=WEEK&perspective=CHART&flags=viral_video_chart&selected_chart=TRACKS&chart_params_id=weekly%3A{formatted_date}%3A{week_date}%3Abr"
        }
        
        # Exceução da requisição com os parâmetros acima e obtenção do json 
        req = requests.post(api_youtube, json=params)
        dados = req.json()

        # O json de saída é aninhado em vários níveis, é necessário 'descer' pelas keys para fazer a normalização apenas da parte de Tracks ('trackviews')
        tracks = json_normalize(dados["contents"]["sectionListRenderer"]["contents"],
                         sep='_',
                         record_path=['musicAnalyticsSectionRenderer','content','trackTypes','trackViews']) 

        # Criação de coluna de data para indicar qual a semana de entrada do registro
        tracks = tracks.assign(data_inicio=current_date)

        # A mesma lógica se aplica para obter os dados de nomes dos artistas, que estão em um nível abaixo no json
        artistas = json_normalize(dados['contents']['sectionListRenderer']['contents'],['musicAnalyticsSectionRenderer','content','trackTypes','trackViews','artists'],
                         sep='_',
                         meta=[['musicAnalyticsSectionRenderer','content','trackTypes','trackViews','id']])

        artistas = artistas.assign(data_inicio=current_date)

        # Final da extração da semana atual, soma-se 7 dias, armazena-se os dados extraídos e avança para a próxima semana no loop
        current_date += timedelta(days=7)

        file_tracks.append(tracks)
        file_artistas.append(artistas)
    
    # Ao final do loop, é feita a concatenação dos dados armazenados de cada semana em dataframes únicos 
    df_artistas = pd.concat(file_artistas)
    df_tracks = pd.concat(file_tracks)

    # Os dataframes são salvos em formato .csv com indicação do início e fim da extração
    end_date = end_date.strftime(date_format) 
    
    artistas_filename = f'artistas_data_{start_date}_{end_date}.csv'
    tracks_filename = f'tracks_data_{start_date}_{end_date}.csv'

    df_artistas.to_csv(artistas_filename)        
    df_tracks.to_csv(tracks_filename)     
    
    return df_artistas, df_tracks   

# Chamada da função. Como não foram passados parâmetros, a função assume os valores default de start_date='20190104' e end_date='20230602'
tracks_data = get_youtube_charts_data() 



C:\Users\rapha\AppData\Local\Temp\ipykernel_4296\2141381167.py:51: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  tracks = json_normalize(dados["contents"]["sectionListRenderer"]["contents"],
C:\Users\rapha\AppData\Local\Temp\ipykernel_4296\2141381167.py:59: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  artistas = json_normalize(dados['contents']['sectionListRenderer']['contents'],['musicAnalyticsSectionRenderer','content','trackTypes','trackViews','artists'],
C:\Users\rapha\AppData\Local\Temp\ipykernel_4296\2141381167.py:51: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  tracks = json_normalize(dados["contents"]["sectionListRenderer"]["contents"],
C:\Users\rapha\AppData\Local\Temp\ipykernel_4296\2141381167.py:59: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  artistas = json_normalize(da

(             kgMid              name  \
 0    /g/11h0d3dnyr     Gabriel Diniz   
 1    /g/11b6_sq0s2        Mc Livinho   
 2    /g/11c54ch2hn    Márcia Fellipe   
 3              NaN       Jerry Smith   
 4    /g/11b60z06t1            DENNIS   
 ..             ...               ...   
 191  /g/11f7j3kh4y             Dj GM   
 192  /g/11frvp2c6j           MC Luck   
 193  /g/11bt_bgy1p           MC Bill   
 194  /g/11jp2p56lt  Nadson O Ferinha   
 195  /g/11jcn2j3bw     Felipe Amorim   
 
     musicAnalyticsSectionRenderer_content_trackTypes_trackViews_id data_inicio  
 0                                        Y:AgWUtFtSviE              2019-01-04  
 1                                        Y:lwp2e0tFtZM              2019-01-04  
 2                                        Y:j47F_UFQHm0              2019-01-04  
 3                                        Y:j47F_UFQHm0              2019-01-04  
 4                                        Y:sSoVIcYmkPY              2019-01-04  
 ..           